# Aquisição de dados no portal da ONS Dados Aberto

https://ons-aws-prod-opendata.s3.amazonaws.com/

https://registry.opendata.aws/ons-opendata-portal/

https://github.com/awslabs/open-data-registry/blob/main/datasets/ons-opendata-portal.yaml#L17

In [45]:
import os
import requests
import xml.etree.ElementTree as ET
from datetime import datetime,  timedelta
import sys
# Adiciona o caminho da pasta src ao sys.path para importar datasets_ons
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))
from datasets_ons import datasets_ons

In [46]:
def listar_arquivos():
    url = "https://ons-aws-prod-opendata.s3.amazonaws.com"
    params = {"list-type": "2", "prefix": "dataset/"}
    arquivos = []
    while True:
        resp = requests.get(url, params=params)
        root = ET.fromstring(resp.content)
        for item in root.findall(".//{*}Contents"):
            key_elem = item.find("{*}Key")
            if key_elem is not None and key_elem.text is not None:
                arquivos.append(key_elem.text)
        token = root.find(".//{*}NextContinuationToken")
        if token is not None and token.text is not None:
            params["continuation-token"] = token.text
        else:
            break
    return arquivos

def listar_arquivos_do_dataset(nome_dataset=None, extensoes=(".csv", ".parquet", ".xlsx", ".pdf")):
    todos = listar_arquivos()
    if nome_dataset is None:
        datasets = sorted({arq.split("/")[1] for arq in todos if arq.startswith("dataset/") and not arq.endswith("/")})
        print(f"\nDatasets disponíveis ({datetime.now().strftime('%d/%m/%y')}):")
        for ds in datasets:
            print("-", ds)
        return

    arquivos = [arq.split("/")[-1] for arq in todos
                if arq.startswith(f"dataset/{nome_dataset}/") and not arq.endswith("/")
                and any(arq.endswith(ext) for ext in extensoes)]

    if not arquivos:
        print(f"Nenhum arquivo encontrado para o dataset '{nome_dataset}'.")
    else:
        print(f"\nArquivos do dataset '{nome_dataset}':")
        for arq in arquivos:
            print("-", arq)

def baixar_arquivo(nome_dataset, nome_arquivo, pasta_destino):
    url = f"https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/{nome_dataset}/{nome_arquivo}"
    os.makedirs(pasta_destino, exist_ok=True)
    caminho = os.path.join(pasta_destino, nome_arquivo)
    resp = requests.get(url)
    if resp.status_code == 200:
        with open(caminho, 'wb') as f:
            f.write(resp.content)
        print(f"Baixado: {nome_arquivo}")
    else:
        print(f"Erro {resp.status_code} ao baixar: {nome_arquivo}")

def baixar_dataset_ons(nome_dataset, pasta_destino, data_inicio=None, data_fim=None):
    """
    Baixa arquivos públicos do ONS a partir do dataset informado, com base na frequência dos dados
    (anual, mensal, diário ou fixo) definida no dicionário datasets_ons.

    Parâmetros:
    ----------
    nome_dataset : str
        Nome do dataset disponível no bucket ONS.
    pasta_destino : str
        Caminho onde os arquivos serão salvos localmente.
    data_inicio : str, opcional
        Data inicial no formato "YYYY", "YYYY-MM" ou "YYYY-MM-DD", dependendo da frequência do dataset.
        Obrigatória para datasets não fixos.
    data_fim : str, opcional
        Data final no formato "YYYY", "YYYY-MM" ou "YYYY-MM-DD", dependendo da frequência do dataset.
        Obrigatória para datasets não fixos.

    Retorna:
    -------
    None
        A função não retorna nada. Os arquivos são salvos localmente na pasta especificada.
    """
    if nome_dataset not in datasets_ons:
        print(f"Dataset '{nome_dataset}' não encontrado.")
        return

    info = datasets_ons[nome_dataset]
    padrao, freq = info['padrao'], info['frequencia']

    if freq == 'fixo':
        baixar_arquivo(nome_dataset, padrao, pasta_destino)
        return

    if not data_inicio or not data_fim:
        print(f"Datas de início e fim são obrigatórias para datasets do tipo '{freq}'.")
        return

    def parse_data(data):
        try:
            return datetime.strptime(data, "%Y-%m-%d")
        except ValueError:
            try:
                return datetime.strptime(data, "%Y-%m")
            except ValueError:
                return datetime.strptime(data, "%Y")

    dt_ini = parse_data(data_inicio)
    dt_fim = parse_data(data_fim)
    nomes = []

    if freq == 'anual':
        for ano in range(dt_ini.year, dt_fim.year + 1):
            nomes.append(padrao.format(ano=ano))

    elif freq == 'mensal':
        atual = datetime(dt_ini.year, dt_ini.month, 1)
        while atual <= dt_fim:
            nomes.append(padrao.format(ano=atual.year, mes=atual.month))
            ano = atual.year + (atual.month // 12)
            mes = atual.month % 12 + 1
            atual = datetime(ano, mes, 1)

    elif freq == 'diario':
        atual = dt_ini
        while atual <= dt_fim:
            nomes.append(padrao.format(ano=atual.year, mes=atual.month, dia=atual.day))
            atual += timedelta(days=1)

    for nome_arq in nomes:
        baixar_arquivo(nome_dataset, nome_arq, pasta_destino)

In [47]:
baixar_dataset_ons("curva-carga-ho", pasta_destino="../data/Curva Carga HO ONS", data_inicio="2000-01-01", data_fim="2025-12-31")

Baixado: CURVA_CARGA_2000.csv
Baixado: CURVA_CARGA_2001.csv
Baixado: CURVA_CARGA_2002.csv
Baixado: CURVA_CARGA_2003.csv
Baixado: CURVA_CARGA_2004.csv
Baixado: CURVA_CARGA_2005.csv
Baixado: CURVA_CARGA_2006.csv
Baixado: CURVA_CARGA_2007.csv
Baixado: CURVA_CARGA_2008.csv
Baixado: CURVA_CARGA_2009.csv
Baixado: CURVA_CARGA_2010.csv
Baixado: CURVA_CARGA_2011.csv
Baixado: CURVA_CARGA_2012.csv
Baixado: CURVA_CARGA_2013.csv
Baixado: CURVA_CARGA_2014.csv
Baixado: CURVA_CARGA_2015.csv
Baixado: CURVA_CARGA_2016.csv
Baixado: CURVA_CARGA_2017.csv
Baixado: CURVA_CARGA_2018.csv
Baixado: CURVA_CARGA_2019.csv
Baixado: CURVA_CARGA_2020.csv
Baixado: CURVA_CARGA_2021.csv
Baixado: CURVA_CARGA_2022.csv
Baixado: CURVA_CARGA_2023.csv
Baixado: CURVA_CARGA_2024.csv
Baixado: CURVA_CARGA_2025.csv
